Importing libraries and function for chunks for 10-folds

In [59]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
import operator
import pandas as pd
import scipy.io as sci
%matplotlib inline
def chunk(xs, n):
    ys = list(xs)
    random.seed(0)
    random.shuffle(ys)
    size = len(ys) // n
    leftovers= ys[size*n:]
    for c in xrange(n):
        if leftovers:
           extra= [ leftovers.pop() ] 
        else:
           extra= []
        yield ys[c*size:(c+1)*size] + extra

In [60]:
def common_neighbours(sub):
    mat = np.array((nx.to_numpy_matrix(sub) != 0) * 1).dot(np.array((nx.to_numpy_matrix(sub) != 0) * 1))
    edgesWithScore = {}
    edges = nx.non_edges(sub)
    for e in edges:
        edgesWithScore[e] = mat[e[0]][e[1]]
    return edgesWithScore 

def salton_index(sub):
    mat = np.array((nx.to_numpy_matrix(sub) != 0) * 1).dot(np.array((nx.to_numpy_matrix(sub) != 0) * 1))
    edgesWithScore = {}
    edges = nx.non_edges(sub)
    for e in edges:
        j = nx.degree(sub, e[0])
        k = nx.degree(sub, e[1])
        if j != 0 and k != 0:
            edgesWithScore[e] = float(mat[e[0]][e[1]])/float(np.sqrt(j * k))
        else:
            edgesWithScore[e] = 0
    return edgesWithScore

def jaccard_index(sub):
    mat = np.array((nx.to_numpy_matrix(sub) != 0) * 1).dot(np.array((nx.to_numpy_matrix(sub) != 0) * 1))
    edgesWithScore = {}
    edges = nx.non_edges(sub)
    for e in edges:
        if nx.degree(sub, e[0]) != 0 or nx.degree(sub, e[1]) != 0:
            edgesWithScore[e] = float(mat[e[0]][e[1]])/float(len(set(sub[e[0]])|set(sub[e[1]])))
        else:
            edgesWithScore[e] = 0
    return edgesWithScore

def sorensen_index(sub):
    mat = np.array((nx.to_numpy_matrix(sub) != 0) * 1).dot(np.array((nx.to_numpy_matrix(sub) != 0) * 1))
    edgesWithScore = {}
    edges = nx.non_edges(sub)
    for e in edges:
        j = nx.degree(sub, e[0])
        k = nx.degree(sub, e[1])
        if j != 0 or k != 0:
            edgesWithScore[e] = 2*float(mat[e[0]][e[1]])/float(j + k)
        else:
            edgesWithScore[e] = 0
    return edgesWithScore

def hub_promoted_index(sub):
    mat = np.array((nx.to_numpy_matrix(sub) != 0) * 1).dot(np.array((nx.to_numpy_matrix(sub) != 0) * 1))
    edgesWithScore = {}
    edges = nx.non_edges(sub)
    for e in edges:
        j = nx.degree(sub, e[0])
        k = nx.degree(sub, e[1])
        if j != 0 and k != 0:
            edgesWithScore[e] = float(mat[e[0]][e[1]])/float(min(j, k))
        else:
            edgesWithScore[e] = 0
    return edgesWithScore

def hub_depressed_index(sub):
    mat = np.array((nx.to_numpy_matrix(sub) != 0) * 1).dot(np.array((nx.to_numpy_matrix(sub) != 0) * 1))
    edgesWithScore = {}
    edges = nx.non_edges(sub)
    for e in edges:
        j = nx.degree(sub, e[0])
        k = nx.degree(sub, e[1])
        if j != 0 or k != 0:
            edgesWithScore[e] = float(mat[e[0]][e[1]])/float(max(j, k))
        else:
            edgesWithScore[e] = 0
    return edgesWithScore

def LHN1_index(sub):
    mat = np.array((nx.to_numpy_matrix(sub) != 0) * 1).dot(np.array((nx.to_numpy_matrix(sub) != 0) * 1))
    edgesWithScore = {}
    edges = nx.non_edges(sub)
    for e in edges:
        j = nx.degree(sub, e[0])
        k = nx.degree(sub, e[1])
        if j != 0 and k != 0:
            edgesWithScore[e] = float(mat[e[0]][e[1]])/float(j * k)
        else:
            edgesWithScore[e] = 0
    return edgesWithScore

def preferential_attachment_index(sub):
    edgesWithScore = {}
    edges = nx.non_edges(sub)
    for e in edges:
        edgesWithScore[e] = nx.degree(sub, e[0]) * nx.degree(sub, e[1])
    return edgesWithScore

def adamic_adar_index(sub):
    edgesWithScore = {}
    edges = nx.non_edges(sub)
    for e in edges:
        edgesWithScore[e] = np.sum(1/np.log(nx.degree(sub, sorted(nx.common_neighbors(sub, e[0], e[1]))).values()))
    return edgesWithScore

def resource_allocation_index(sub):
    edgesWithScore = {}
    edges = nx.non_edges(sub)
    for e in edges:
        edgesWithScore[e] = np.sum(1/np.array(
                nx.degree(sub, sorted(nx.common_neighbors(sub, e[0], e[1]))).values()).astype(float))
    return edgesWithScore

def katz_index(sub):
    mat = np.array((nx.to_numpy_matrix(sub) != 0) * 1.)
    ide = np.identity(len(mat))
    beta = (1/float(max(np.linalg.eigh(mat)[0])))/2
    sim = np.linalg.inv(ide - beta*mat) - ide
    edgesWithScore = {}
    edges = nx.non_edges(sub)
    for e in edges:
        edgesWithScore[e] = sim[e[0]][e[1]]
    return edgesWithScore

def lhn2_index(sub, phi = 0.1):
    mat = np.array((nx.to_numpy_matrix(sub) != 0) * 1.)
    ide = np.identity(len(mat))
    dma = np.diagflat(mat.sum(axis = 1))
    edgesWithScore = {}
    edges = nx.non_edges(sub)
    for e in edges:
        edgesWithScore[e] = 0
    if np.linalg.det(dma) != 0:
        lambd = float(max(np.linalg.eigh(mat)[0]))
        sim = (2 * sub.number_of_edges() * lambd * np.linalg.inv(dma)).dot(np.linalg.inv(
        ide - (phi/lambd) * mat)).dot(np.linalg.inv(dma))
        edges = nx.non_edges(sub)
        for e in edges:
            edgesWithScore[e] = sim[e[0]][e[1]]
    return edgesWithScore

def act_index(sub):
    mat = np.array((nx.to_numpy_matrix(sub) != 0) * 1.)
    dma = np.diagflat(mat.sum(axis = 1))
    sim = np.linalg.pinv(dma - mat)
    edgesWithScore = {}
    edges = nx.non_edges(sub)
    for e in edges:
        if sim[e[0]][e[0]] + sim[e[1]][e[1]] - 2*sim[e[0]][e[1]] != 0:
            edgesWithScore[e] = 1/float(sim[e[0]][e[0]] + sim[e[1]][e[1]] - 2*sim[e[0]][e[1]])
        else:
            edgesWithScore[e] = 0
    return edgesWithScore

def cbl_index(sub):
    mat = np.array((nx.to_numpy_matrix(sub) != 0) * 1.)
    dma = np.diagflat(mat.sum(axis = 1))
    sim = np.linalg.pinv(dma - mat)
    edgesWithScore = {}
    edges = nx.non_edges(sub)
    for e in edges:
        if sim[e[0]][e[0]] * sim[e[1]][e[1]] != 0:
            edgesWithScore[e] = float(sim[e[0]][e[1]])/np.sqrt(sim[e[0]][e[0]] * sim[e[1]][e[1]])
        else:
            edgesWithScore[e] = 0
    return edgesWithScore

def rwr_index(sub, c = 0.5):
    mat = np.array((nx.to_numpy_matrix(sub) != 0) * 1.)
    dma = np.diagflat(mat.sum(axis = 1))
    sim = np.linalg.pinv(dma - mat)
    ide = np.identity(len(mat))
    pmt = np.identity(len(mat))
    es = []
    for a in sub.nodes():
        e = np.zeros(len(mat))
        e[a] = 1
        es.append(e)
        for b in sub.nodes():
            if not nx.has_path(sub, source = a, target = b):
                pmt[a][b] = 0
            else:
                if nx.degree(sub, a) != 0:
                    pmt[a][b] = 1/float(nx.degree(sub, a))
                else:
                    pmt[a][b] = 0
    es = np.array(es)
    qs = []
    for a in sub.nodes():
        if np.linalg.det(ide - c*np.transpose(pmt)) != 0:
            q = ((1 - c)*np.linalg.inv(ide - c*np.transpose(pmt))).dot(es[a])
        else:
            q = es[a]
        qs.append(q)
    qs = np.array(qs)
    edgesWithScore = {}
    edges = nx.non_edges(sub)
    for e in edges:
        edgesWithScore[e] = qs[e[0]][e[1]] + qs[e[1]][e[0]]
    return edgesWithScore

def matrix_forest_index(sub, alpha = 1):
    mat = np.array((nx.to_numpy_matrix(sub) != 0) * 1.)
    dma = np.diagflat(mat.sum(axis = 1))
    ide = np.identity(len(mat))
    sim = np.linalg.inv(ide + alpha*(dma - mat))
    edgesWithScore = {}
    edges = nx.non_edges(sub)
    for e in edges:
        edgesWithScore[e] = sim[e[0]][e[1]]
    return edgesWithScore

def calculate_aucs(G, nfolds = 10):
    df = pd.DataFrame()
    random.seed(0)
    folds = [i for i in chunk(G.edges(), nfolds)]
    aucs = []
    for i in xrange(nfolds):
        sub = G.copy()
        for c in folds[i]:
            sub.remove_edge(*c)
        complement = nx.non_edges(sub)
        edgesWithScore = [common_neighbours(sub), 
                           salton_index(sub),
                           jaccard_index(sub),
                           sorensen_index(sub),
                           hub_promoted_index(sub),
                           hub_depressed_index(sub),
                           LHN1_index(sub),
                           preferential_attachment_index(sub),
                           adamic_adar_index(sub),
                           resource_allocation_index(sub),
                           katz_index(sub),
                           lhn2_index(sub),
                           act_index(sub),
                           cbl_index(sub),
                           rwr_index(sub),
                           matrix_forest_index(sub)]
        auc = []
        for indn in xrange(len(edgesWithScore)):
            highScore = 0
            sameScore = 0
            allScore = 0
            for ne in nx.non_edges(G):
                for e in folds[i]:
                    if edgesWithScore[indn][ne] < edgesWithScore[indn][e]:
                        highScore += 1
                    elif edgesWithScore[indn][ne] == edgesWithScore[indn][e]:
                        sameScore += 1
                    allScore += 1
            auc.append(float(highScore + 0.5*sameScore)/float(allScore))
        aucs.append(auc)
    #print aucs
    auc_means = np.mean(np.array(aucs), axis = 0)
    return {'CN' : auc_means[0],
            'SaI' : auc_means[1],
            'JI' : auc_means[2],
            'SoI' : auc_means[3],
            'HPI' : auc_means[4],
            'HDI' : auc_means[5],
            'LHN1' : auc_means[6],
            'PAI' : auc_means[7],
            'AAI' : auc_means[8],
            'RAI' : auc_means[9],
            'KI' : auc_means[10],
            'LHN2' : auc_means[11],
            'ACT' : auc_means[12],
            'CBL' : auc_means[13],
            'RWR' : auc_means[14],
            'MFI' : auc_means[15]}

In [65]:
df = pd.DataFrame(columns = ['CN', 'SaI', 'JI', 'SoI', 
                             'HPI', 'HDI', 'LHN1', 'PAI', 
                             'AAI', 'RAI', 'KI', 'LHN2', 
                             'ACT', 'CBL', 'RWR', 'MFI'])
pd.set_option('precision',4)

In [66]:
df.loc['karate club'] = predict_nodes(nx.karate_club_graph())
newman_adjnoun = nx.read_gml('./netws/newman/adjnoun/adjnoun.gml')
df.loc['newman adjnoun'] = predict_nodes(newman_adjnoun)

In [68]:
df

,CN,SaI,JI,SoI,HPI,HDI,LHN1,PAI,AAI,RAI,KI,LHN2,ACT,CBL,RWR,MFI
karate club,0.700,0.636,0.607,0.607,0.712,0.593,0.600,0.712,0.726,0.733,0.755,0.613,0.666,0.739,0.618,0.749
newman adjnoun,0.662,0.606,0.603,0.603,0.618,0.603,0.566,0.744,0.662,0.659,0.714,0.516,0.742,0.589,0.738,0.667


In [71]:
print df[['CN', 'SaI', 'JI', 'SoI', 
                             'HPI', 'HDI', 'LHN1', 'PAI', 
                             'AAI', 'RAI',]].to_latex()

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &     CN &    SaI &     JI &    SoI &    HPI &    HDI &   LHN1 &    PAI &    AAI &    RAI \\
\midrule
karate club    &  0.700 &  0.636 &  0.607 &  0.607 &  0.712 &  0.593 &  0.600 &  0.712 &  0.726 &  0.733 \\
newman adjnoun &  0.662 &  0.606 &  0.603 &  0.603 &  0.618 &  0.603 &  0.566 &  0.744 &  0.662 &  0.659 \\
\bottomrule
\end{tabular}



In [55]:
predict_nodes(nx.karate_club_graph())

{'AAI': 0.72559523809523818,
 'ACT': 0.66597900029577051,
 'CBL': 0.73873669032830525,
 'CN': 0.69990757172434193,
 'HDI': 0.59264086069210298,
 'HPI': 0.71227632357290749,
 'JI': 0.60682860100561964,
 'KI': 0.7551907719609583,
 'LHN1': 0.59992236024844714,
 'LHN2': 0.61252033422064478,
 'MFI': 0.74868012422360253,
 'PAI': 0.71162562851227462,
 'RAI': 0.73339618456078082,
 'RWR': 0.61805124223602481,
 'SaI': 0.63614315291333923,
 'SoI': 0.60682860100561964}

[array([ 0.72269319,  0.66199355,  0.65669865,  0.65669865,  0.67068988,
        0.65076321,  0.61040588,  0.76450627,  0.72112902,  0.7151976 ,
        0.75476983,  0.5       ,  0.76479943,  0.6558252 ,  0.75819736,
        0.7018248 ]), array([ 0.61427114,  0.5779879 ,  0.58148571,  0.58148571,  0.59116994,
        0.58374262,  0.56602065,  0.69834105,  0.6190741 ,  0.6210238 ,
        0.67066579,  0.5       ,  0.6698084 ,  0.53732134,  0.67139065,
        0.63542466]), array([ 0.71159939,  0.64825732,  0.64928337,  0.64928337,  0.65425701,
        0.65368475,  0.59705919,  0.78886243,  0.71878175,  0.71791232,
        0.75357913,  0.5       ,  0.79358106,  0.595238  ,  0.78362174,
        0.7009875 ]), array([ 0.65769659,  0.59391277,  0.58682278,  0.58682278,  0.61510443,
        0.58250172,  0.55636051,  0.77485714,  0.65418874,  0.65103228,
        0.7202897 ,  0.55324421,  0.74952513,  0.58374061,  0.75096481,
        0.67039673]), array([ 0.67339255,  0.61074522,  0.6044845 ,  

{'AAI': 0.66163687093789125,
 'ACT': 0.7421744714800701,
 'CBL': 0.58867712586434096,
 'CN': 0.66214340406400662,
 'HDI': 0.60282738633075761,
 'HPI': 0.61840145370111688,
 'JI': 0.60344266306234151,
 'KI': 0.71380368743418066,
 'LHN1': 0.56567697364432867,
 'LHN2': 0.51574740886543879,
 'MFI': 0.66749749917435941,
 'PAI': 0.74384260488981924,
 'RAI': 0.65907147609237449,
 'RWR': 0.73765957524114723,
 'SaI': 0.60584940296672207,
 'SoI': 0.60344266306234151}